In [1]:
from tqdm import tqdm
import numpy as np
from src.structures import User, Movie
from src.data_methods import read_movies,read_viewers
import kagglehub
import os

In [2]:
# Download latest version
#please ikke offentliggør min api key. 
os.environ['KAGGLE_USERNAME'] = "marcusgaleajacobsen"
os.environ['KAGGLE_KEY'] = "32a3003f52c97053841ea46c492128dc"
datapath = kagglehub.dataset_download("netflix-inc/netflix-prize-data")

In [3]:
n_lines = np.inf #number of reviews to read
datafiles = ["combined_data_1.txt"]#, "combined_data_2.txt", "combined_data_3.txt", "combined_data_4.txt"]
with_tqdm = True #set to True to see progress bar (reduce speed)
reviews_pr_user = 10 #number of reviews per user to read

movies = read_movies(datapath)
users = read_viewers(datapath, movies, datafiles = datafiles, with_tqdm= with_tqdm, n_lines=n_lines,)# reviews_pr_user=reviews_pr_user) #read only 100000 

100%|██████████| 24058263/24058263 [01:16<00:00, 315829.28it/s]


In [4]:
from src.similarity_methods import compute_signatures, create_buckets, get_candidates, trim_candidates
n_hashes = 20
n_buckets = len(movies)**2

SIG = compute_signatures(movies, n_hashes, with_tqdm = True)

  0%|          | 0/4499 [00:00<?, ?it/s]

100%|██████████| 4499/4499 [04:15<00:00, 17.61it/s]


In [5]:
b = 5
n_buckets = 2**16
buckets = create_buckets(SIG, bands = b, n_buckets= n_buckets)

In [6]:
candidates = get_candidates(buckets)

100%|██████████| 5/5 [00:00<00:00, 123.40it/s]


In [7]:
#binomial coefficient
from math import comb
n_candidates = len(candidates)
n_movies = len(movies)
n_pairs = comb(n_movies, 2)
n_pairs

10118251

In [8]:
r = n_hashes // b
t = (1/b)**(1/r)

In [9]:
n_removed = trim_candidates(movies, candidates, threshold = 0.3)
print(f"Removed {n_removed} from {len(candidates)} candidates")

100%|██████████| 1007/1007 [00:08<00:00, 125.34it/s]

Removed 936 from 1007 candidates


In [ ]:
#compute similarity matrix
sim_matrix = np.zeros((n_movies, n_movies))
for i, movie_id1 in enumerate(tqdm(movies)):
    for j, movie_id2 in enumerate(movies):
        sim_matrix[i, j] = movies[movie_id1.similarity(movies[movie_id2], method = "jaccard")]

4499